# Kohya ss py3.11


# 启动Webui

注意，如果你使用多卡训练，请先配置accelerate（会用多卡的相信你会配置了）
```bash
accelerate config
```
本镜像在RTX4090单卡上配置，如果训练中报错，也可以尝试配置acclerate


# 运行下面的代码块从云端更新最新的`start.ipynb`（本文档）
### 更新完后请重新刷新jupyterlab的文件管理然后打开此文档 `/root/start.ipynb`
### 现在运行的版本是 230803.2

In [ ]:
!rm -rf /root/start.ipynb && wget https://drive.t4wefan.pub/d/autodl/sdxl/latest/start.ipynb

# 下载sdxl 1.0 base 模型(默认已下载到/root/ckpts/sd_xl_base_1.0_0.9vae.safetensors)
## 也可以手动下载

In [ ]:
# 先安装openssl
!apt update && apt install openssl
!source /etc/network_turbo #启动学术加速

In [ ]:
!wget --no-check-certificat --max-redirect=3 -P /root/autodl-tmp https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0_0.9vae.safetensors
!unset http_proxy && unset https_proxy

In [ ]:
# 默认模型已下载，在/root/ckpts/sd_xl_base_1.0_0.9vae.safetensors 可以直接启动（因为这个脚本需要使用多个端口，默认已加上share，可以使用gradio来控制ui，镜像内自带frp，可以使用你自己的隧道保证数据安全）
!mkdir /root/autodl-tmp/current #创建临时文件夹存储训练中数据
!cd /root/kohya_ss && ./gui.sh --server_port 28000 --listen 0.0.0.0 --headless --share

## 也可以使用公开服务端口，使用下面这一行（这会导致无法启动tensorboard）

In [8]:
# 默认模型已下载，在/root/ckpts/sd_xl_base_1.0_0.9vae.safetensors
!mkdir /root/autodl-tmp/current #创建临时文件夹存储训练中数据
!cd /root/kohya_ss && ./gui.sh --server_port 6006 --listen 0.0.0.0 --headless

mkdir: cannot create directory ‘/root/autodl-tmp/current’: File exists
venv folder does not exist. Not activating...
16:49:44-919646 INFO     Version: v21.8.5                                       
                                                                                
16:49:44-930666 INFO     nVidia toolkit detected                                
16:49:46-045017 INFO     Torch 2.0.1+cu117                                      
16:49:46-062791 INFO     Torch backend: nVidia CUDA 11.7 cuDNN 8500             
16:49:46-077803 INFO     Torch detected GPU: NVIDIA GeForce RTX 4090 VRAM 24217 
                         Arch (8, 9) Cores 128                                  
16:49:46-079252 INFO     Verifying modules instalation status from              
                         /root/kohya_ss/requirements_linux.txt...               
16:49:46-083124 INFO     Verifying modules instalation status from              
                         requirements.txt...                             

## 启动 Stable-diffusion-webui 来调试模型(适合新手，默认安装tagger可以用来给训练集打标，默认6006端口，也可以改成自己的)

In [9]:
!cd /root/stable-diffusion-webui && python launch.py --port 6006 --xformers --enable-insecure-extension-access --listen --no-half

Python 3.10.8 (main, Nov 24 2022, 14:13:03) [GCC 11.2.0]
Version: v1.4.1-201-g14cf434b
Commit hash: 14cf434bc36d0ef31f31d4c6cd2bd15d7857d5c8
Installing requirements
loading WD14-tagger reqs from /root/stable-diffusion-webui/extensions/stable-diffusion-webui-wd14-tagger/requirements.txt
Checking WD14-tagger requirements.

Launching Web UI with arguments: --port 6006 --xformers --enable-insecure-extension-access --listen --no-half
2023-08-03 16:50:08.713699: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-03 16:50:09.466242: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
== WD14 tagger gpu, uname_result(system='Linux', node='autodl-container-264d4da021-c22db0f8', release='5.4.0-153-generic', version=

## 启动 ComfyUI 来调试模型(适合专业用户，默认6006端口，也可以改成自己的)

In [10]:
!cd /root/ComfyUI && python main.py --preview-method auto --port 6006 --listen 0.0.0.0

Total VRAM 24217 MB, total RAM 515689 MB
xformers version: 0.0.20
2023-08-03 16:50:26.965655: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-03 16:50:27.710874: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Set vram state to: NORMAL_VRAM
Device: cuda:0 NVIDIA GeForce RTX 4090 : cudaMallocAsync
Using xformers cross attention
Starting server

To see the GUI go to: http://0.0.0.0:6006
^C

Stopped server




## 手动部署（本镜像自带环境，理论上可以直接运行，推荐py3.11）

In [ ]:
# 如果你同时使用多个gpu
!export MKL_SERVICE_FORCE_INTEL=1
!export MKL_THREADING_LAYER=GNU

克隆仓库

In [ ]:
# 启动学术加速
!source /etc/network_turbo

In [ ]:
!git clone -b sdxl-dev https://github.com/bmaltais/kohya_ss.git

安装pytorch（根据当前镜像最高版本来选择）

In [ ]:
#cu117
#!conda install pytorch torchvision torchaudio pytorch-cuda=11.7 -c pytorch -c nvidia

#cu118
!conda install pytorch torchvision torchaudio pytorch-cuda=11.8 -c pytorch -c nvidia

安装需要的依赖

In [ ]:
!cd /root/konya_ss
!pip install -r requirements.txt
!pip install xformers==0.0.20 bitsandbytes==0.35.0 tensorboard==2.12.3 tensorflow==2.12.0

如果你使用上一个代码块时依赖不全，尝试这个

In [ ]:
!pip install accelerate==0.19.0 albumentations==1.3.0 altair==4.2.2 dadaptation==3.1 diffusers[torch]==0.18.2 easygui==0.98.3 einops==0.6.0 fairscale==0.4.13 ftfy==6.1.1 gradio==3.36.1 huggingface-hub==0.15.1 lion-pytorch==0.0.6 lycoris_lora==1.8.0 invisible-watermark==0.2.0 open-clip-torch==2.20.0 opencv-python==4.7.0.68 prodigyopt==1.0 pytorch-lightning==1.9.0 rich==13.4.1 safetensors==0.3.1 timm==0.6.12 tk==0.1.0 toml==0.10.2 transformers==4.30.2 voluptuous==0.13.1 wandb==0.15.0 xformers==0.0.20 bitsandbytes==0.35.0 tensorboard==2.12.3 tensorflow==2.12.0

In [ ]:
# 记得取消学术加速
!unset http_proxy && unset https_proxy



# 一些附赠软件

### frp 方便用于内网穿透，需要手写配置文件，相信你已经会了
### gohttpserver 轻量级文件管理web，可以方便的打包和多线程下载文件

In [ ]:
# 启动 gohttpserver 会占用6006端口，你也可以换成自己的端口
!cd /root && ./gohttpserver --port 6006